In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
# !pip install array-to-latex

from os.path import exists
import pandas as pd
import evaluate 
import json
from semantic_group.jointEM import *
import openai
import heapq
from tqdm import tqdm 
import os
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
metric = evaluate.load("rouge")
from tqdm import tqdm
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.util import ngrams
import string
import random
import openai
import evaluate
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from difflib import SequenceMatcher
metric = evaluate.load("rouge")

stopword_set = set(stopwords.words('english'))
%load_ext autoreload
%autoreload 2

In [ ]:
from openicl import PromptTemplate 
template_map = {
    "samsum": [
        '</E>Dialogue:"\n </dialogue>" \nSummarize the above dialogue: </summary>',
        {"dialogue": "</dialogue>", "summary": "</summary"},
    ],
    "virattt/financial-qa-10K": [
        '</E>Context: "\n </context>"\n\nQuestion: </question>\nAnswer: </answer>',
        {"context": "</context>", "question": "</question>", "answer": "</answer>"},
    ],
    "Malikeh1375/medical-question-answering-datasets": [
        "</E>Question: </question>\n\nAnswer: </answer>\n",
        {"input": "</question>", "output": "</answer>"},
    ],
}
data_name = "Malikeh1375/medical-question-answering-datasets"
template = PromptTemplate(
    template_map[data_name][0],
    template_map[data_name][1],
    ice_token="</E>",
)

In [ ]:
test_num = 100
eps_half = 0.13
eps_1 = 0.23
eps_3 = 0.63
eps_5 = 0.93
eps_8 = 1.32
eps_12 = 1.92
eps_inf = np.inf
openai.api_key = "YOUR OPENAI API KEY"
config = {
    "model": "gpt-3.5-turbo", 
    "max_token": 200, 
    "temperature": 0.02 
}

In [ ]:
def selectwith_public1shot(public_example, query, candidates):
    public_ref =  public_example.split("Question:")[-2]
    prompt = '['
    for candidate in candidates:
        prompt += candidate + ',\n'
    prompt = prompt[:-2] + ']'
    prompt = prompt + '\nThe answer is:'
    query = query + "\nPick the most accurate answer for the question with the following answer candidates ranked by their freqeuncy from high to low: "
    context = "Question:\n"+public_ref + "Question:\n"+query
    final_prompt = context + prompt

    return final_prompt  

In [ ]:
def top_k_indices(counts, k):
    # Get the indices of the k largest elements
    return heapq.nlargest(k, range(len(counts)), key=lambda i: counts[i])

In [ ]:
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from rouge_score import rouge_scorer

def calculate_all_scores(predictions, references):
    """Calculate BLEU, METEOR, and ROUGE-1 scores for each prediction-reference pair."""
    # Initialize the scorer for ROUGE-1
    rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
    
    # Smoothing function for BLEU
    smoothie = SmoothingFunction().method4

    # Lists to store the results for each metric
    bleu_scores = []
    meteor_scores = []
    rouge1_scores = []

    # Loop through each prediction-reference pair
    for pred, ref in zip(predictions, references):
        # Tokenize both prediction and reference
        pred_tokens = word_tokenize(pred)
        ref_tokens = word_tokenize(ref)

        # BLEU score calculation
        bleu_score = sentence_bleu([ref_tokens], pred_tokens, weights=(1.0, 0.0, 0.0, 0.0), smoothing_function=smoothie)
        bleu_scores.append(bleu_score)

        # METEOR score calculation (expects strings or lists of tokens)
        meteor_score_value = meteor_score([ref_tokens], pred_tokens)
        meteor_scores.append(meteor_score_value)

        # ROUGE-1 score calculation
        rouge_score = rouge_scorer_instance.score(ref, pred)
        rouge1_score = rouge_score['rouge1'].fmeasure
        rouge1_scores.append(rouge1_score)

    # Return the results as a dictionary of lists
    return {
        'BLEU': np.mean(bleu_scores),
        'METEOR': np.mean(meteor_scores),
        'ROUGE1': np.mean(rouge1_scores)
    }


In [ ]:
def calculate_mean_and_std_from_dict_scores(score_list):
    """
    Calculate the mean and standard deviation for BLEU, Levenshtein similarity, and ROUGE-1 scores
    from a list of dictionaries where each dictionary contains {'BLEU': value, 'Levenshtein': value, 'ROUGE1': value}.
    
    Parameters:
    score_list (list): A list of dictionaries, where each dictionary contains 'BLEU', 'Levenshtein', 'ROUGE1' scores.
    
    Returns:
    dict: A dictionary containing the mean and standard deviation for BLEU, Levenshtein, and ROUGE-1 scores.
    """
    # Unpack the scores into separate lists for BLEU, Levenshtein, and ROUGE-1
    bleu_scores = [scores['BLEU'] for scores in score_list]
    meteor_scores = [scores['METEOR'] for scores in score_list]
    rouge1_scores = [scores['ROUGE1'] for scores in score_list]
    
    # Calculate mean and standard deviation for each metric
    results = {}
    
    # BLEU
    bleu_mean = np.mean(bleu_scores) * 100
    bleu_std = np.std(bleu_scores) * 100
    results['bleu'] = {'mean': bleu_mean, 'std': bleu_std}
    
    # Levenshtein
    meteor_mean = np.mean(meteor_scores) * 100
    meteor_std = np.std(meteor_scores) * 100
    results['meteor'] = {'mean': meteor_mean, 'std': meteor_std}
    
    # ROUGE-1
    rouge1_mean = np.mean(rouge1_scores) * 100
    rouge1_std = np.std(rouge1_scores) * 100
    results['rouge1'] = {'mean': rouge1_mean, 'std': rouge1_std}
    
    return results



In [ ]:
def load_histogram(path):
    with open(path ,'r') as f:
        file = json.load(f)
        histograms =[]
        for line in file:
            histograms.append(line)
    return histograms

In [ ]:
path = "semantic_group/0.95_medical-question-answering-datasets_chatdoctor_icliniq_gpt-3.5-turbo_100way-4shot_aggregate.json"
#path = "/home/jsh/workspace/DP-ICL/Generate_seperate_reponses/long_generation/semantic_group/oodpublic_0.92_medical-question-answering-datasets_chatdoctor_icliniq_gpt-3.5-turbo_100way-4shot_aggregate.json"
private_path = "semantic_group/0.95_medical-question-answering-datasets_chatdoctor_icliniq_gpt-3.5-turbo_100way-4shot_private.json"
histograms = load_histogram(path)
private_histograms = load_histogram(private_path)

In [ ]:
output_private = "/home/jsh/workspace/DP-ICL/Generate_seperate_reponses/long_generation/output/medical-question-answering-datasets_chatdoctor_icliniq_gpt-3.5-turbo_100way-4shot_private.json"
output_public = "/home/jsh/workspace/DP-ICL/Generate_seperate_reponses/long_generation/output/medical-question-answering-datasets_chatdoctor_icliniq_gpt-3.5-turbo_100way-4shot_public.json"
#output_public = "output/ood_medical-question-answering-datasets_chatdoctor_healthcaremagic_gpt-3.5-turbo_100way-4shot_public.json"
with open(output_private, 'r') as f:
    file_private = json.load(f)
with open(output_public, 'r') as f:
    file_public = json.load(f)

### KSA Baseline

In [ ]:
# get predictions from the json file
trials = 1
ds_size = 100 
ensemble= 100 
private_only = True      
eps = eps_inf
k=40  
final_preds_group =[]
references_group = []
print(f"eps={eps}, k={k}, trials={trials}")
for i in range(trials):
     random.seed(2*i)
     np.random.seed(2*i)
     final_preds = []
     references =[]
     for i in tqdm(range(ds_size)):
          all_tokens = {} # key: token, value: count
          for j in range(ensemble):
               sentence_private, sentence_public = file_private[i*ensemble+j]['prediction'], file_public[i*ensemble+j]['prediction']
               tokens_private = nltk.word_tokenize(sentence_private)
               tokens_public = nltk.word_tokenize(sentence_public)
               tokens_group = (tokens_private, tokens_public) if not private_only else [tokens_private]
               for tokens in tokens_group:
                    onegrams = set(ngrams(tokens, 1))
                    # onegrams = set(onegrams)
                    # making onegrams a set to avoid duplicate tokens
                    for token in onegrams:
                         # only add one gram for one sentence
                         if token in all_tokens:
                              all_tokens[token] += 1
                         else:
                              all_tokens[token] = 1
          all_tokens_sorted = sorted(all_tokens.items(), key=lambda x: x[1], reverse=True)
          # ignore those non-words tokens
          filtered_tokens = {}
          for token, count in all_tokens_sorted:
               if not all(word in string.punctuation for word in token) and token[0] not in stopword_set:
                    filtered_tokens[token] = count
          filtered_tokens_sorted = sorted(filtered_tokens.items(), key=lambda x: x[1], reverse=True)
          item_counts = np.array([count for token, count in filtered_tokens_sorted])
          k=min(k, len(item_counts))
          if eps==np.inf:
               joint_out = top_k_indices(item_counts, k)
          else:
               joint_out = joint(item_counts, k, epsilon=eps, neighbor_type=1)
          filtered_tokens = np.array(filtered_tokens_sorted, dtype=object)[joint_out]
          filtered_tokens = [token_tuple[0][0] for token_tuple in filtered_tokens]
          prompt = '['
          for token in filtered_tokens:
               prompt += token + ', '
          prompt = prompt[:-2] + ']'
          prompt = prompt + '\nThe answer is:'
          public_ref =  file_public[i*ensemble+j]['prompt'].split("Question:")[-2]
          query =  file_public[i*ensemble+j]['prompt'].split("Question:")[-1]
          zero_shot_sentence = "Question:\n"+public_ref + "Question:\n"+query.replace('Answer:','Answer the above question with the following word suggestions ranked by their frequency from high to low:') 
          final_prompt = zero_shot_sentence  + prompt
          response = openai.ChatCompletion.create(
          model=config["model"],  # Use "gpt-3.5-turbo" or your chosen model
          messages=[
               {
                    "role": "user",
                    "content": final_prompt,
               },  # The prompt provided by the user
          ],
          max_tokens=config["max_token"],
          temperature=config["temperature"],
          stop=["\n"],
          )["choices"][0]["message"]["content"]
          references.append(file_private[i*ensemble+j]['reference'])
          final_preds.append(response)
     final_preds_group.append(final_preds)
     references_group.append(references)

In [ ]:
scores = [] 
for i in range(trials):
    scores.append(calculate_all_scores(final_preds_group[i], references_group[i]))
calculate_mean_and_std_from_dict_scores(scores)

### SGA

In [ ]:
trials = 1
k= 6
ds_size = 100
ensemble= 100 
eps=eps_inf 
eps_over_query = [eps_8]
test_num=1
#k_value = [2,3,5,6,8]
#histograms = private_histograms
for eps in eps_over_query:
    final_preds_group =[]
    references_group = []
    print(f"eps={eps}, k={k}, trials={trials}")
    for i in range(trials):
        random.seed(2*i)
        np.random.seed(2*i)
        predictions = []
        references = []
        for j in tqdm(range(test_num)): 
            # private top-k selection 
            cnts = np.array(histograms[j]['count'])
            top_k = min(k,len(cnts))
            if eps==np.inf:
                joint_out = top_k_indices(cnts, top_k)
            else:
                joint_out = joint(cnts, k=top_k, epsilon=eps, neighbor_type=1)
            candidates = [] 
            for idx in joint_out:
                keys = list(histograms[j].keys())[:-3]
                target_key = keys[idx]
                candidates.append(histograms[j][target_key][0][0])          # pick representative 

            query = histograms[j]['query']
            reference = histograms[j]['reference']
            if k>1:
                prompt = selectwith_public1shot(file_public[j*ensemble]['prompt'], query, candidates)
                response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",  # Use "gpt-3.5-turbo" or your chosen model
                messages=[
                    {
                            "role": "user",
                            "content": prompt,
                    },  # The prompt provided by the user
                ],
                max_tokens=200,
                temperature=0.02,
                stop=["\n"],
                )["choices"][0]["message"]["content"]
            else:
                response = candidates[0]    # Direct top-1 
            prediction = response
            predictions.append(prediction)
            references.append(reference)
        final_preds_group.append(predictions)
        references_group.append(references)
    scores = [] 
    for i in range(trials):
        scores.append(calculate_all_scores(final_preds_group[i], references_group[i]))
    results = calculate_mean_and_std_from_dict_scores(scores)
    print(results)
    

In [ ]:
scores = [] 
for i in range(trials):
    scores.append(calculate_all_scores(final_preds_group[i], references_group[i]))
calculate_mean_and_std_from_dict_scores(scores)

In [ ]:
query


### Direct top-1

In [ ]:
references_group[0][0]

In [ ]:
trials = 1
ds_size = 100 
final_preds_group =[]
references_group = []
k=3
eps=eps_1
print(f"eps={eps}, trials={trials}")
for trial in range(trials):
    predictions = []
    references = []
    for j in tqdm(range(test_num)): 
        # Private top-1 selection 
        cnts = np.array(histograms[j]['count'])
        k = min(k,len(cnts))
        if eps==np.inf:
            joint_out = top_k_indices(cnts, k=k)
        else:
            joint_out = joint(cnts, k=k, epsilon=eps, neighbor_type=1)
        keys = list(histograms[j].keys())[:-3]
        target_key = keys[joint_out[0]]
        prediction = histograms[j][target_key][0][0]          # pick representative 
        reference = histograms[j]['reference']
        predictions.append(prediction)
        references.append(reference)
    final_preds_group.append(predictions)
    references_group.append(references)

In [ ]:
scores = [] 
for i in range(trials):
    scores.append(calculate_all_scores(final_preds_group[i], references_group[i]))
calculate_mean_and_std_from_dict_scores(scores)

In [ ]:
final_preds_group[0]

### 4-shot ($\varepsilon=\infty$)

In [ ]:
path = "output/medical-question-answering-datasets_chatdoctor_icliniq_gpt-3.5-turbo_100way-4shot_private.json"
trials=5
num_ensemble = 100 
final_preds_group =[]
references_group = []
with open(path, 'r') as f:
    file = json.load(f)
num_examples = len(file)

for i in range(trials):
    predictions = []
    references = []
    for j in range(0, num_examples, ensemble):
        example = file[j+i] 
        prediction, reference = example["prediction"], example["reference"]
        predictions.append(prediction)
        references.append(reference)
    final_preds_group.append(predictions)
    references_group.append(references)    
        
    

In [ ]:
scores = [] 
for i in range(trials):
    scores.append(calculate_all_scores(final_preds_group[i], references_group[i]))
calculate_mean_and_std_from_dict_scores(scores)

### 0-shot

In [ ]:
path = "output/medical-question-answering-datasets_chatdoctor_icliniq_gpt-3.5-turbo_5way-0shot_private.json"
trials=5
num_ensmble = 5 
with open(path , 'r') as f:
    file = json.load(f)
num_examples = len(file)
final_preds_group =[]
references_group = []
for i in range(trials):
    predictions = []
    references = []
    for j in range(0, num_examples, ensemble):
        example = file[j+i] 
        prediction, reference = example["prediction"], example["reference"]
        predictions.append(prediction)
        references.append(reference)
    final_preds_group.append(predictions)
    references_group.append(references)   

In [ ]:
scores = [] 
for i in range(trials):
    scores.append(calculate_all_scores(final_preds_group[i], references_group[i]))
calculate_mean_and_std_from_dict_scores(scores)

### With OOD Public 

In [ ]:
output_public = "output/ood_medical-question-answering-datasets_chatdoctor_healthcaremagic_gpt-3.5-turbo_100way-4shot_public.json"
with open(output_public, 'r') as f:
    file_public = json.load(f)

In [ ]:
path = "semantic_group/oodpublic_0.92_medical-question-answering-datasets_chatdoctor_icliniq_gpt-3.5-turbo_100way-4shot_aggregate.json"
private_path = "semantic_group/oodpublic_0.92_medical-question-answering-datasets_chatdoctor_icliniq_gpt-3.5-turbo_100way-4shot_aggregate.json"
histograms = load_histogram(path)
private_histograms = load_histogram(private_path)

In [ ]:
trials = 1
k=2
ds_size = 100
ensemble= 100 
final_preds_group =[]
references_group = []
eps=0.01
private_only = False 
histograms = private_histograms if private_only else histograms
print(f"eps={eps}, private={private_only}, trials={trials}")
for i in range(trials):
    random.seed(2*i)
    np.random.seed(2*i)
    predictions = []
    references = []
    for j in tqdm(range(test_num)): 
        # private top-k selection 
        cnts = np.array(histograms[j]['count'])
        top_k = min(k,len(cnts))
        if eps==np.inf:
            joint_out = top_k_indices(cnts, top_k)
        else:
            joint_out = joint(cnts, k=top_k, epsilon=eps, neighbor_type=1)
        candidates = [] 
        for idx in joint_out:
            keys = list(histograms[j].keys())[:-3]
            target_key = keys[idx]
            candidates.append(histograms[j][target_key][0][0])          # pick representative 

        query = histograms[j]['query']
        reference = histograms[j]['reference']
        prompt = selectwith_public1shot(file_public[j*ensemble]['prompt'], query, candidates)

        response = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",  # Use "gpt-3.5-turbo" or your chosen model
          messages=[
               {
                    "role": "user",
                    "content": prompt,
               },  # The prompt provided by the user
          ],
          max_tokens=200,
          temperature=0.02,
          stop=["\n"],
          )["choices"][0]["message"]["content"]
        prediction = response
        predictions.append(prediction)
        references.append(reference)
    final_preds_group.append(predictions)
    references_group.append(references)

In [ ]:
scores = [] 
for i in range(trials):
    scores.append(calculate_all_scores(final_preds_group[i], references_group[i]))
calculate_mean_and_std_from_dict_scores(scores)

### Hyperparmeter analysis (#ensemble, #quries)

In [ ]:
# e=(10,30,50,75,100) q=(1, 10, 100, 1000, 10000)
eps_3_10 = 4.29 
eps_3_25 = 1.84
eps_3_50 = 1.15
eps_3_100 = 0.63